## Take a look

In [1]:
import os
import time

data_dir = 'USPTO_DIVERSE'
phase = ["train", "val", "test"]
for data in phase:
    # fp = f"./USPTO_miniFULL/src-{data}.txt"
    fp = f"./{data_dir}/src-{data}.txt"
    with open(fp, 'r') as f:
        lines = f.readlines()
        print(f'{data} samples: {len(lines)}')
print(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()))

fp = f"./{data_dir}/src-test.txt"
with open(fp, 'r') as f:
    lines = f.readlines()
    prod_set = set(lines)
    print(f'Unique test products: {len(prod_set)}')

FileNotFoundError: [Errno 2] No such file or directory: './USPTO_DIVERSE/src-train.txt'

## 1. Split old csv and create 1-to-N dataset

In [2]:
import numpy as np
import os
import pandas as pd
from tqdm import tqdm
import csv
from pandas import DataFrame

import sys
sys.path.append("..")
from utils.data_utils import canonicalize_smiles


data_dir = 'USPTO_DIVERSE'
# csv_path = f"./USPTO_miniFULL/USPTO_FULL.csv"
csv_path = f"./{data_dir}/USPTO-MIT.csv"
ids=[]
with open(csv_path, "r") as csv_file:
    df = pd.read_csv(csv_file)
    df['reactant'] = df["reactions"].map(lambda x: x.split(">")[0])
    df['product'] = df["reactions"].map(lambda x: x.split(">")[-1])
    sorted_df = df.sort_values(by=['product'], ascending=True)
    last_p, last_r, last_id = None, None, None
    for idx, idf in tqdm(sorted_df.iterrows()):
        smi_p, smi_r = idf['product'], idf['reactant']
        if '.' in smi_p:
            continue
        smi_p = canonicalize_smiles(smi_p, remove_atom_number=True)
        if last_p == smi_p and last_r == smi_r:
            continue
        elif last_p == smi_p:
            if last_id not in ids:
                ids += [last_id,idx]
            else:
                ids += [idx]
        last_p, last_r, last_id = smi_p, smi_r, idx

## Record index
print(len(ids))
type(ids)
full_ids = np.array(ids)
t = full_ids

# print(df.head())
newdf = DataFrame()
newdf["reactants>reagents>production"] = df['reactions'][t[:]]
print(newdf.head())
csv_path = f'./{data_dir}/fromMIT.csv'
newdf.to_csv(csv_path)
print(f'new dataframe saving ...')


## Split and save
# fn = f"./USPTO_miniFULL/fromFULL.csv"
# fn = f"./miniFULL/fromFULL.csv"
with open(csv_path, "r") as csv_file:
    df = pd.read_csv(csv_file)
    df['product'] = df["reactants>reagents>production"].map(lambda x: x.split(">")[-1])
    # df = df.reindex(np.random.permutation(df.index))
    df = df.reindex(np.arange(len(df.index)))
    ptr = []
    last_sp = None
    for i, sp in tqdm(enumerate(df['product'])):
        if sp != last_sp:
            ptr.append(i)
            last_sp = sp
    ptr.append(i+1)
    pre = ptr[:-1]
    post = ptr[1:]
    train_ids, val_ids, test_ids = [], [], []
    for id in range(len(pre)):
        if (id+1)%8 == 0:
            val_ids += list(np.arange(pre[id], post[id]))
        elif (id+1)%9 == 0:
            test_ids += list(np.arange(pre[id], post[id]))
        else:
            train_ids += list(np.arange(pre[id], post[id]))

    # import ipdb; ipdb.set_trace()
    per_num = len(df) // 10
    train_df, val_df, test_df = DataFrame(), DataFrame(), DataFrame()
    train_df["reactants>reagents>production"] = df["reactants>reagents>production"][train_ids]
    val_df["reactants>reagents>production"] = df["reactants>reagents>production"][val_ids]
    test_df["reactants>reagents>production"] = df["reactants>reagents>production"][test_ids]

    train_df.to_csv(f'./{data_dir}/raw_train.csv')
    val_df.to_csv(  f'./{data_dir}/raw_val.csv')
    test_df.to_csv( f'./{data_dir}/raw_test.csv')
    print(f'Split done!')

479035it [03:09, 2530.91it/s]


28849
                            reactants>reagents>production
328721   CCOC(=O)C(Br)C(=O)OCC.CCOC(=O)C(C)NCc1ccccc1>>Br
159167               C=CCBr.C=CCOC1CC(C)(C)NC(C)(C)C1>>Br
380846  BrB(Br)Br.CN(C)CC1=CCCc2c1ccc(O)c2O>CO.COc1ccc...
311806  CCN(C(C)C)C(C)C.CCOC(=O)CCCCBr>CC(N)Cc1ccccc1....
104639  N=c1sc2cc(OC(F)(F)F)ccc2n1CCO.OCCBr>CCO.Nc1nc2...
new dataframe saving ...


28849it [00:00, 2625615.19it/s]


Split done!


## 2. Raw csv To txt

In [3]:
import csv
import os
from tqdm import tqdm
import sys
sys.path.append("..")
from utils.data_utils import canonicalize_smiles, tokenize_smiles

import time

def split_raw_csv(fn, ofn_src, ofn_tgt):
    print(f"Converting {fn} into {ofn_src} and {ofn_tgt}")
    with open(fn, "r") as csv_file, open(ofn_src, "w") as of_src, open(ofn_tgt, "w") as of_tgt:
        csv_reader = csv.DictReader(csv_file)
        last_p = None
        for row in tqdm(csv_reader):
            smi = row["reactants>reagents>production"]
            smi_r, _, smi_p = smi.split(">")
            if '.' in smi_p or canonicalize_smiles(smi_p, remove_atom_number=True) == 'CC':
                continue
            smi_p, smi_r = canonicalize_smiles(smi_p, remove_atom_number=True), canonicalize_smiles(smi_r, remove_atom_number=True)
            if smi_p != last_p:
                last_rs = []
            if last_p == smi_p and smi_r in last_rs:
                continue
            last_p = smi_p
            last_rs.append(smi_r)

            of_src.write(f"{tokenize_smiles(smi_p)}\n")
            of_tgt.write(f"{tokenize_smiles(smi_r)}\n")


def main():
    fp = f"./{data_dir}"
    for phase in ["train", "val", "test"]:
    # for phase in ["test"]:
        fn = os.path.join(fp, f"raw_{phase}.csv")
        ofn_src = os.path.join(fp, f"src-{phase}.txt")
        ofn_tgt = os.path.join(fp, f"tgt-{phase}.txt")
        split_raw_csv(fn, ofn_src, ofn_tgt)
    split_raw_csv(fn, ofn_src, ofn_tgt)

if __name__ == "__main__":
    main()
    print(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()))

Converting ./USPTO_DIVERSE/raw_train.csv into ./USPTO_DIVERSE/src-train.txt and ./USPTO_DIVERSE/tgt-train.txt


22207it [00:14, 1539.75it/s]


Converting ./USPTO_DIVERSE/raw_val.csv into ./USPTO_DIVERSE/src-val.txt and ./USPTO_DIVERSE/tgt-val.txt


3898it [00:02, 1648.17it/s]


Converting ./USPTO_DIVERSE/raw_test.csv into ./USPTO_DIVERSE/src-test.txt and ./USPTO_DIVERSE/tgt-test.txt


2744it [00:01, 1511.11it/s]


Converting ./USPTO_DIVERSE/raw_test.csv into ./USPTO_DIVERSE/src-test.txt and ./USPTO_DIVERSE/tgt-test.txt


2744it [00:01, 1561.86it/s]

2022-05-11 14:01:18


## Make Answer JSON from CSV

In [4]:
import csv
import os
from tqdm import tqdm
import sys
sys.path.append("..")
from utils.data_utils import canonicalize_smiles, tokenize_smiles

import time


def split_raw_csv(fn, save_dir):
    print(f"Converting {fn} into answer dict in {save_dir}")
    with open(fn, "r") as csv_file:
        answer = {}
        csv_reader = csv.DictReader(csv_file)
        last_p = None
        for idx, row in tqdm(enumerate(csv_reader)):
            smi = row["reactants>reagents>production"]
            smi_r, _, smi_p = smi.split(">")
            if '.' in smi_p or canonicalize_smiles(smi_p, remove_atom_number=True) == 'CC':
                continue
            smi_p, smi_r = canonicalize_smiles(smi_p, remove_atom_number=True), canonicalize_smiles(smi_r, remove_atom_number=True)
            if smi_p != last_p:
                last_rs = []
            if last_p == smi_p and smi_r in last_rs:
                continue
            prod_str, reac_str = tokenize_smiles(smi_p), tokenize_smiles(smi_r)
            if last_p != smi_p:
                answer[prod_str] = {'product':prod_str, 'reaction':0, 'reactant':[]}
            answer[prod_str]['reactant'].append(reac_str)
            answer[prod_str]['reaction']+=1
            last_p = smi_p
            last_rs.append(smi_r)

        import json
        json_str = json.dumps(answer, indent=4)
        with open(fn+'.json', 'w') as json_file:
            json_file.write(json_str)
        print(f'JSON Done!')

def main():
    # fp = "./miniFULL"
    # fp = "./diverse"
    for phase in ["test","val"]:#
        fn = os.path.join(fp, f"raw_{phase}.csv")
        split_raw_csv(fn, fp)

if __name__ == "__main__":
    main()
    print(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()))

Converting ./diverse/raw_test.csv into answer dict in ./diverse


0it [00:00, ?it/s]RDKit WARNING: [10:08:00] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [10:08:00] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [10:08:00] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [10:08:00] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [10:08:00] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [10:08:00] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [10:08:00] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [10:08:00] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [10:08:00] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [10:08:00] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [10:08:00] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [10:08:00] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [10:08:

JSON Done!
Converting ./diverse/raw_val.csv into answer dict in ./diverse


0it [00:00, ?it/s]RDKit WARNING: [10:08:04] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [10:08:04] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [10:08:04] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [10:08:04] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [10:08:04] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [10:08:04] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [10:08:04] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [10:08:04] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [10:08:04] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [10:08:04] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [10:08:04] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [10:08:04] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [10:08:

JSON Done!
2021-12-22 10:08:09


## 3. Make Answer JSON from TXT

In [4]:
import csv
import os
from tqdm import tqdm
import sys
import time


def creatJSON(data_path, src_file, tgt_file):
    with open(src_file, 'r') as sf, open(tgt_file, 'r') as tf:
        src = sf.readlines()
        tgt = tf.readlines()
        answer = {}
        last_p = None
        for idx, row in tqdm(enumerate(src)):
            smi_p = row.strip()
            smi_r = tgt[idx].strip()
            if smi_p != last_p:
                last_rs = []
            if last_p == smi_p and smi_r in last_rs:
                continue
            if last_p != smi_p:
                answer[smi_p] = {'product':smi_p, 'reaction':0, 'reactant':[]}
            answer[smi_p]['reactant'].append(smi_r)
            answer[smi_p]['reaction']+=1
            last_p = smi_p
            last_rs.append(smi_r)
        import json
        json_str = json.dumps(answer, indent=4)
        if 'val' in src_file:
            fn = os.path.join(data_path, "raw_val.csv")
        elif 'test' in src_file:
            fn = os.path.join(data_path, "raw_test.csv")
        elif 'train' in src_file:
            fn = os.path.join(data_path, "raw_train.csv")
        with open(fn+'.json', 'w') as json_file:
            json_file.write(json_str)
        print(f'JSON Done!')

def main():
    fp = f"./{data_dir}"
    # fp = "./USPTO_50k"
    for phase in ["test","val"]:
    # for phase in ["train"]:
        src_file, tgt_file = os.path.join(fp, f"src-{phase}.txt"), os.path.join(fp, f"tgt-{phase}.txt")
        creatJSON(fp, src_file, tgt_file)

if __name__ == "__main__":
    main()
    print(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()))

2527it [00:00, 505412.53it/s]


JSON Done!


3235it [00:00, 586571.56it/s]

JSON Done!
2022-05-11 14:03:32


In [4]:
smi = 'O=C(Cl)C(=O)Cl.O=C(O)c1ccc2cncc(Br)c2n1>>[Cl-]'
smi_r, _, smi_p = smi.split(">")
a=canonicalize_smiles(smi_p, remove_atom_number=True)
b=tokenize_smiles(canonicalize_smiles(smi_p, remove_atom_number=True))
print(canonicalize_smiles(smi_p, remove_atom_number=True))
print(tokenize_smiles(canonicalize_smiles(smi_p, remove_atom_number=True)))


CC
C C
